In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt


In [2]:
# Đọc dữ liệu từ các file CSV
train_data = pd.read_csv("train.csv")  # Dữ liệu huấn luyện
val_data = pd.read_csv("val.csv")  # Dữ liệu kiểm thử

# Kiểm tra dữ liệu
print(train_data.head())
print(val_data.head())


                                                text  label
0                  sp tốt giao tầm tuần tại hàng qte      0
1    mn nên mua nha hình_ảnh mang tính_chất minh_hoạ      0
2        h h d d s s d d h s h d d s s h s h s s d d      0
3  đã nhận đx hàng dép đẹp giá rẻ nhưng về dép bị...      0
4                                   chất_liệu cao_su      0
                                                text  label
0                         đúng với mô_tả đúng vs ảnh      0
1                                             tạm ổn      0
2  túi giống với hình đăng , có đều kích_thước tú...      0
3  y y t x t cuc c t c x t d x t xtret t g g te e...      0
4                          độ bền màu om m chưa test      0


In [3]:
X_train = train_data['text'].values
y_train = train_data['label'].values

X_val = val_data['text'].values
y_val = val_data['label'].values


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
max_len = 128  # Giới hạn độ dài câu

# Tokenize và encode cho cả tập huấn luyện và kiểm thử
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=max_len)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=max_len)


In [5]:
# Chuyển dữ liệu thành TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(len(X_train)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_val
)).batch(16)


In [6]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 nhãn


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])


In [8]:
epochs = 3  # Số epoch có thể thay đổi tùy theo yêu cầu
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


Epoch 1/3


 49/234 [=====>........................] - ETA: 12:52 - loss: 0.6486 - accuracy: 0.6454

KeyboardInterrupt: 

In [9]:
results = model.evaluate(val_dataset)
print("Test Loss, Test Accuracy:", results)


26/26 [==============================] - 23s 805ms/step - loss: 0.6060 - accuracy: 0.6851
Test Loss, Test Accuracy: [0.60602205991745, 0.6850961446762085]


In [13]:
import tkinter as tk
from tkinter import messagebox
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Tải lại mô hình và tokenizer từ thư mục đã lưu
save_path = "./saved_model"
model = TFBertForSequenceClassification.from_pretrained(save_path)
tokenizer = BertTokenizer.from_pretrained(save_path)


# Tải mô hình và tokenizer
# model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Hàm dự đoán từ mô hình
def predict_sentiment(text):
    # Tokenize văn bản đầu vào
    encoding = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")

    # Dự đoán với mô hình
    predictions = model.predict(encoding)
    logits = predictions.logits

    # Chuyển logits thành xác suất
    probabilities = tf.nn.softmax(logits, axis=-1)
    predicted_label = tf.argmax(probabilities, axis=-1).numpy()[0]

    return "Hài lòng" if predicted_label == 1 else "Không hài lòng"

# Hàm xử lý khi người dùng nhấn nút "Dự đoán"
def on_predict():
    # Lấy văn bản người dùng nhập
    user_input = entry.get()

    # Nếu không có nhập liệu
    if not user_input:
        messagebox.showwarning("Cảnh báo", "Vui lòng nhập lời nhận xét!")
        return

    # Dự đoán thái độ cảm nhận của người dùng
    result = predict_sentiment(user_input)

    # Hiển thị kết quả
    messagebox.showinfo("Kết quả dự đoán", f"Lời nhận xét của bạn: {user_input}\n\nKết quả: {result}")

# Tạo giao diện Tkinter
root = tk.Tk()
root.title("Chatbot Dự Đoán Thái Độ Hài Lòng")

# Cài đặt kích thước cửa sổ
root.geometry("400x300")

# Label hướng dẫn người dùng
label = tk.Label(root, text="Nhập nhận xét về sản phẩm:", font=("Arial", 12))
label.pack(pady=10)

# Entry cho người dùng nhập nhận xét
entry = tk.Entry(root, font=("Arial", 12), width=40)
entry.pack(pady=10)

# Nút để dự đoán
predict_button = tk.Button(root, text="Dự đoán", font=("Arial", 12), command=on_predict)
predict_button.pack(pady=10)

# Chạy giao diện Tkinter
root.mainloop()


Some layers from the model checkpoint at ./saved_model were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


1/1 [==============================] - 0s 45ms/step


In [12]:
# Lưu mô hình và tokenizer vào thư mục 'saved_model'
save_path = "./saved_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json')

In [17]:
from flask import Flask, render_template, request, jsonify
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Tải mô hình và tokenizer
save_path = "./saved_model"
try:
    # Thử tải mô hình từ thư mục đã lưu
    model = TFBertForSequenceClassification.from_pretrained(save_path)
    tokenizer = BertTokenizer.from_pretrained(save_path)
except:
    # Nếu không tìm thấy, tải mô hình mặc định và lưu lại
    model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

# Khởi tạo Flask app
app = Flask(__name__)

# Hàm dự đoán
def predict_sentiment(text):
    # Tokenize văn bản đầu vào
    encoding = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")
    predictions = model.predict(encoding)
    logits = predictions.logits
    probabilities = tf.nn.softmax(logits, axis=-1)
    predicted_label = tf.argmax(probabilities, axis=-1).numpy()[0]
    return "Hài lòng" if predicted_label == 1 else "Không hài lòng"

# Route chính (trang giao diện)
@app.route("/")
def index():
    return render_template("index.html")

# API xử lý dự đoán
@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    user_input = data.get("text", "")
    if not user_input.strip():
        return jsonify({"error": "Vui lòng nhập lời nhận xét!"}), 400
    result = predict_sentiment(user_input)
    return jsonify({"result": result})

if __name__ == "__main__":
    app.run(debug=True, port=5001)


Some layers from the model checkpoint at ./saved_model were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [16]:
%tb

SystemExit: 1